# Churn Prediction Pipeline
Este notebook implementa um pipeline completo de previsão de churn usando PySpark no Databricks.

## Estrutura:
- Ingestão (Silver Layer)
- Transformação e Feature Engineering (Gold Layer)
- Treinamento de Modelo (Random Forest)
- Avaliação (Acurácia e AUC-ROC)
- Saída Final (churn_predictions_gold)

## 1. Ingestão de Dados (Silver Layer)
Carregamos os dados da camada Silver a partir do arquivo CSV.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, expr

spark = SparkSession.builder.appName("ChurnPredictionPipeline").getOrCreate()

# Caminho do dataset Silver
silver_path = "/Volumes/workspace/voc/churn/churn_silver_2025.csv"

df_silver = spark.read.option("header", True).option("inferSchema", True).csv(silver_path)
df_silver.printSchema()
df_silver.show(5)

## 2. Transformação e Feature Engineering (Gold Layer)
Criamos novas features, como a taxa de uso por valor pago.

In [ ]:
# Feature Engineering: taxa de uso por valor pago
df_gold = df_silver.withColumn("taxa_uso_valor", col("total_logs_app_30d") / col("valor_mensalidade"))

# Conversão da coluna alvo para inteiro
df_gold = df_gold.withColumn("label", col("churn").cast("integer"))
df_gold.show(5)

## 3. Preparação dos Dados para Machine Learning
Usamos StringIndexer para variáveis categóricas e VectorAssembler para juntar todas as features.

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

# Indexação da variável categórica
indexer = StringIndexer(inputCol="categoria_principal_voc", outputCol="categoria_index")

# Seleção de features
feature_cols = ["valor_mensalidade", "total_logs_app_30d", "tickets_suporte_abertos", "score_sentimento_voc", "taxa_uso_valor", "categoria_index"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

pipeline = Pipeline(stages=[indexer, assembler])
df_ml = pipeline.fit(df_gold).transform(df_gold)
df_ml.select("id_cliente", "features", "label").show(5)

## 4. Treinamento do Modelo (Random Forest)
Dividimos os dados em treino e teste e treinamos um Random Forest Classifier.

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

# Split treino/teste
train, test = df_ml.randomSplit([0.7, 0.3], seed=42)

# Modelo Random Forest
rf = RandomForestClassifier(featuresCol="features", labelCol="label", probabilityCol="probability", predictionCol="prediction")
model = rf.fit(train)

# Predições
predictions = model.transform(test)
predictions.select("id_cliente", "probability", "prediction").show(5)

## 5. Avaliação do Modelo
Calculamos Acurácia e AUC-ROC.

In [ ]:
# Acurácia
evaluator_acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator_acc.evaluate(predictions)

# AUC-ROC
evaluator_auc = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = evaluator_auc.evaluate(predictions)

print(f"Acurácia: {accuracy}")
print(f"AUC-ROC: {auc}")

## 6. Saída Final (Gold Layer)
Geramos o DataFrame final `churn_predictions_gold` com id_cliente, probabilidade de churn e predição final.

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# Extrair probabilidade de churn (classe 1)
extract_prob = udf(lambda v: float(v[1]), DoubleType())
churn_predictions_gold = predictions.withColumn("prob_churn", extract_prob(col("probability"))) \
    .select("id_cliente", "prob_churn", "prediction")

churn_predictions_gold.show(10)

# ✅ Conclusão
Neste notebook, implementamos um pipeline completo de previsão de churn:
- Ingestão da camada Silver
- Transformação e feature engineering para camada Gold
- Treinamento com Random Forest
- Avaliação com Acurácia e AUC-ROC
- Geração do DataFrame final `churn_predictions_gold`

Este notebook está pronto para ser executado no Databricks.